In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
H, W = 114, 129

ref_y, ref_x = torch.meshgrid(
                torch.linspace(
                    0.5, H - 0.5, H, dtype=torch.float32),
                torch.linspace(
                    0.5, W - 0.5, W, dtype=torch.float32))
                    # 둘 다 (H=114, W=129)
                    # [  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000] 이 114줄

In [15]:
ref_x

tensor([[  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000],
        [  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000],
        [  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000],
        ...,
        [  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000],
        [  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000],
        [  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000]])

In [16]:
ref_y = ref_y.reshape(-1)[None] #tensor([[  0.5000,   0.5000,   0.5000,  ..., 113.5000, 113.5000, 113.5000]]) -> torch.Size([1, 14706])
ref_x = ref_x.reshape(-1)[None] #tensor([[  0.5000,   1.5000,   2.5000,  ..., 126.5000, 127.5000, 128.5000]]) -> torch.Size([1, 14706])
ref = torch.stack((ref_x, ref_y), -1) # torch.Size([1, 14706, 2])
"""
tensor([[[  0.5000,   0.5000],
         [  1.5000,   0.5000],
         [  2.5000,   0.5000],
         ...,
         [126.5000, 113.5000],
         [127.5000, 113.5000],
         [128.5000, 113.5000]]])
"""

'\ntensor([[[  0.5000,   0.5000],\n         [  1.5000,   0.5000],\n         [  2.5000,   0.5000],\n         ...,\n         [126.5000, 113.5000],\n         [127.5000, 113.5000],\n         [128.5000, 113.5000]]])\n'

In [18]:
ref_list = []
ref_list.append(ref)
ref_list.append(ref)

In [29]:
reference_points = torch.cat(ref_list, 1)
reference_points.shape

torch.Size([1, 29412, 2])

In [30]:
valid_ratios = torch.randint(1, 10, (2, 4, 2))
print(reference_points[:, :, None].shape)
reference_points = reference_points[:, :, None] * valid_ratios[:, None]

torch.Size([1, 29412, 1, 2])


In [24]:
reference_points[:, :, None].shape

torch.Size([2, 29412, 1, 4, 2])

In [34]:
spatial_shapes = torch.Tensor([[114, 129], [57, 65], [29, 33], [15, 17]])
print(spatial_shapes.shape)
level_start_index = torch.cat((spatial_shapes.new_zeros((1, )), spatial_shapes.prod(1).cumsum(0)[:-1]))

torch.Size([4, 2])


In [35]:
level_start_index

tensor([    0., 14706., 18411., 19368.])

In [3]:
embed_dims = 256
cls_out_channels = 2 # 1일 수도?
num_pred = 4 # 디코더 레이어 개수 +1
fc_cls = nn.Linear(embed_dims, cls_out_channels)
cls_branches = nn.ModuleList([fc_cls for _ in range(num_pred)]) # len = 4

In [9]:
output_proposals = torch.randint(0, 9, (2, 10000, 4)) #(bs, num_keys, 4)
output_proposals[..., 1:2].shape

torch.Size([2, 10000, 1])

In [32]:
enc_outputs_class = torch.randint(0, 9, (3, 10000, 2)) #(bs, num_keys, cls_out_channels)
topk_proposals = torch.topk(enc_outputs_class[..., 0], 300, dim=1)[0]
enc_outputs_kpt_unact = torch.randint(0, 1, (3, 10000, 34)) #(bs, num_keys, 2*17)

In [33]:
topk_proposals.unsqueeze(-1).repeat(1, 1, 34).shape

torch.Size([3, 300, 34])

In [39]:

topk_kpts_unact = torch.gather(
                enc_outputs_kpt_unact, 1,
                topk_proposals.unsqueeze(-1).repeat(
                    1, 1, enc_outputs_kpt_unact.size(-1)))

In [41]:
topk_kpts_unact.sigmoid().shape

torch.Size([3, 300, 34])

In [51]:
num_qurey = 300
c = 256
query_embed = torch.randint(0, 2, (num_qurey, c))
query_pos, query = torch.split(query_embed, c, dim=1)
query_pos.shape, query.shape

(torch.Size([300, 255]), torch.Size([300, 1]))

In [60]:
batch_size, input_img_h, input_img_w = 2, 120, 240
feat = torch.randint(0, 256, (168, 320), dtype=torch.float)
img_masks = feat.new_ones((batch_size, input_img_h, input_img_w))
mlvl_mask = F.interpolate(img_masks[None], size=feat.shape[-2:]).to(torch.bool).squeeze(0)
mlvl_mask.shape

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])


torch.Size([2, 168, 320])